# All material ©2019, Alex Siegman

---

## Welcome to Projects in Programming & Data Science. We're going to jump right in to the mix. 

### Today we are going to leverage the CitiBike API to populate a MySQL database at regular intervals. Consider this your warm-up for the semester. 

---

## MySQL Setup 
#### (https://www.mysql.com/about/)

**To install MySQL, in your terminal:**

1. \> brew install mysql
2. \> unset TMPDIR
3. \> mysql_secure_installation
4. \> {enter your preferred password}
5. \> {skip all options presented}

**To run MySQL, in your terminal:**

1. \> mysql -u root -p
2. \> {enter password}
3. \> quit

**To set MySQL privileges:** 

1. \> mysql -u root -p
2. \> {enter password}
3. \> grant all privileges on *.* to 'username'@'localhost' identified by 'password' 

> _to change your password, use mysql> alter user 'user'@'localhost' identified by 'newpassword'_

4. \> flush privileges
5. \> quit

**To create a MySQL database:**

1. mysql> create database citibike 

## Now, let's check out the API we'll be working with:

### https://streamdata.io/developers/api-gallery/new-york-citibike-api/

In [1]:
# first, let's request the json from the CitiBike API URL

import time 
import json 
import urllib.request

with urllib.request.urlopen("https://gbfs.citibikenyc.com/gbfs/en/station_status.json") as url:
    data = json.loads(url.read().decode())
    print(data)

{'last_updated': 1565108291, 'ttl': 10, 'data': {'stations': [{'station_id': '168', 'num_bikes_available': 35, 'num_ebikes_available': 0, 'num_bikes_disabled': 1, 'num_docks_available': 11, 'num_docks_disabled': 0, 'is_installed': 1, 'is_renting': 1, 'is_returning': 0, 'last_reported': 1565108198, 'eightd_has_available_keys': False, 'eightd_active_station_services': [{'id': 'bedaaf2b-8664-469e-8681-26ff8059765b'}]}, {'station_id': '281', 'num_bikes_available': 7, 'num_ebikes_available': 0, 'num_bikes_disabled': 6, 'num_docks_available': 53, 'num_docks_disabled': 0, 'is_installed': 1, 'is_renting': 1, 'is_returning': 0, 'last_reported': 1565108289, 'eightd_has_available_keys': True, 'eightd_active_station_services': [{'id': '32461582-cd1e-4ecf-a5ea-563593fa7009'}]}, {'station_id': '304', 'num_bikes_available': 6, 'num_ebikes_available': 0, 'num_bikes_disabled': 0, 'num_docks_available': 27, 'num_docks_disabled': 0, 'is_installed': 1, 'is_renting': 1, 'is_returning': 0, 'last_reported': 

In [2]:
stations = data['data']['stations'] # iterate through the json to find the station data

In [3]:
import pandas as pd # we'll use pandas just to visualize our data

df_stations = pd.DataFrame(stations)
df_stations.head() # check the first five station entries

,eightd_active_station_services,eightd_has_available_keys,is_installed,is_renting,is_returning,last_reported,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,num_ebikes_available,station_id
0,[{'id': 'bedaaf2b-8664-469e-8681-26ff8059765b'}],False,1,1,0,1565108198,35,1,11,0,0,168
1,[{'id': '32461582-cd1e-4ecf-a5ea-563593fa7009'}],True,1,1,0,1565108289,7,6,53,0,0,281
2,[{'id': 'a58d9e34-2f28-40eb-b4a6-c8c01375657a'}],False,1,1,0,1565108038,6,0,27,0,0,304
3,[{'id': '8ec29d39-9642-466a-9a20-aad1e5c4788a'}],False,1,1,0,1565108241,4,1,32,0,0,337
4,[{'id': '6a00be67-3935-4ada-a20d-8007831afb85'}],False,1,1,0,1565108065,12,0,23,0,0,347


In [4]:
import MySQLdb 

# MySQLdb is a way to interface with a MySQL database via a Python API 
# http://mysql-python.sourceforge.net/MySQLdb.html

In [5]:
db = MySQLdb.connect(passwd="password",db="citibike") # connect to our db

In [6]:
c = db.cursor() # set our cursor

In [7]:
# create our table

c.execute("CREATE TABLE IF NOT EXISTS StationInfo (station_id int, num_ebikes_available int, num_docks_disabled int, num_docks_available int, num_bikes_disabled int, num_bikes_available int, last_reported varchar(250), is_returning int, is_renting int, is_installed int, eightd_has_available_keys bool);")

c.close()
db.commit()

In [8]:
from datetime import datetime # import the datetime library 

c = db.cursor() # set our cursor 

# below, we are going to populate our table using "INSERT IGNORE INTO"
# this allows us to avoid errors when we periodically update our database 

query_template = """INSERT IGNORE INTO StationInfo(station_id, num_ebikes_available, num_docks_disabled, \
num_docks_available, num_bikes_disabled, num_bikes_available, last_reported, is_returning, is_renting, \
is_installed, eightd_has_available_keys) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""

for entry in stations: # for every station entry in the json 
    station_id = int(entry['station_id']) # find and set station_id
    num_ebikes_available = int(entry['num_ebikes_available']) # find and set num_ebikes_available
    num_docks_disabled = int(entry['num_docks_disabled']) # find and set num_docks_disabled
    num_docks_available = int(entry['num_docks_available']) # find and set num_docks_available
    num_bikes_disabled = int(entry['num_bikes_disabled']) # find and set num_bikes_disabled
    num_bikes_available = int(entry['num_bikes_available']) # find and set num_bikes_available
    last_reported = str(entry['last_reported']) # find and set last_reported
    is_returning = int(entry['is_returning']) # find and set is_returning
    is_renting = int(entry['is_renting']) # find and set is_renting
    is_installed = int(entry['is_installed']) # find and set is_installed
    eightd_has_available_keys = bool(entry['eightd_has_available_keys']) # find and set eightd_has_available_keys
                           
    print("Inserting Station:", station_id, num_ebikes_available, num_docks_disabled, num_docks_available, num_bikes_disabled, num_bikes_available, last_reported, is_returning, is_renting, is_installed, eightd_has_available_keys) 
    query_parameters = (station_id, num_ebikes_available, num_docks_disabled, num_docks_available, num_bikes_disabled, num_bikes_available, last_reported, is_returning, is_renting, is_installed, eightd_has_available_keys) 
   
    c.execute(query_template, query_parameters) # execute insert query and parameters

c.close()
db.commit()

Inserting Station: 168 0 0 11 1 35 1565108198 0 1 1 False
Inserting Station: 281 0 0 53 6 7 1565108289 0 1 1 True
Inserting Station: 304 0 0 27 0 6 1565108038 0 1 1 False
Inserting Station: 337 0 0 32 1 4 1565108241 0 1 1 False
Inserting Station: 347 0 0 23 0 12 1565108065 0 1 1 False
Inserting Station: 359 0 0 53 2 9 1565107973 0 1 1 False
Inserting Station: 377 0 0 36 0 11 1565108183 1 1 1 False
Inserting Station: 388 0 0 32 2 1 1565108288 0 1 1 False
Inserting Station: 402 0 0 29 1 9 1565107841 0 1 1 False
Inserting Station: 426 0 0 28 0 1 1565108159 0 1 1 True
Inserting Station: 456 0 0 28 3 4 1565108047 0 1 1 False
Inserting Station: 469 0 0 43 2 12 1565107713 0 1 1 False
Inserting Station: 491 0 0 51 0 0 1565107753 0 1 1 False
Inserting Station: 3092 0 0 19 0 8 1565107849 0 1 1 False
Inserting Station: 3459 0 0 31 0 3 1565108250 0 1 1 False
Inserting Station: 3461 0 0 23 1 9 1565108240 0 1 1 False
Inserting Station: 3467 0 0 20 0 22 1565107963 0 1 1 False
Inserting Station: 72 0 

Inserting Station: 400 0 0 12 2 1 1565108114 1 1 1 False
Inserting Station: 401 0 0 38 1 3 1565107897 1 1 1 False
Inserting Station: 403 0 0 25 0 6 1565107994 1 1 1 False
Inserting Station: 405 0 0 28 1 22 1565108256 1 1 1 True
Inserting Station: 406 0 0 28 0 6 1565108181 1 1 1 False
Inserting Station: 408 0 0 20 1 2 1565107754 1 1 1 False
Inserting Station: 410 0 0 44 0 1 1565108141 1 1 1 False
Inserting Station: 411 0 7 22 2 0 1565107203 1 1 1 False
Inserting Station: 412 0 0 12 4 13 1565107798 1 1 1 False
Inserting Station: 414 0 0 1 0 0 1565032577 0 0 0 False
Inserting Station: 415 0 0 9 0 33 1565108017 1 1 1 False
Inserting Station: 416 0 0 28 0 3 1565106394 1 1 1 False
Inserting Station: 417 0 0 1 2 22 1565108144 1 1 1 False
Inserting Station: 418 0 0 1 1 21 1565108207 1 1 1 False
Inserting Station: 419 0 0 13 1 9 1565107111 1 1 1 False
Inserting Station: 420 0 0 22 1 0 1565108285 1 1 1 False
Inserting Station: 421 0 0 11 0 8 1565105906 1 1 1 False
Inserting Station: 422 0 0 75 0

Inserting Station: 3094 0 0 15 0 3 1565108120 1 1 1 False
Inserting Station: 3095 0 0 20 0 6 1565103903 1 1 1 False
Inserting Station: 3096 0 0 34 1 4 1565106361 1 1 1 False
Inserting Station: 3100 0 0 40 0 11 1565108016 1 1 1 False
Inserting Station: 3101 0 0 18 0 9 1565108272 1 1 1 False
Inserting Station: 3102 0 0 27 0 0 1565101682 1 1 1 False
Inserting Station: 3105 0 0 19 1 4 1565103900 1 1 1 False
Inserting Station: 3106 0 0 21 0 1 1565108039 1 1 1 False
Inserting Station: 3107 0 0 5 0 34 1565107885 1 1 1 False
Inserting Station: 3108 0 0 51 0 0 1565107533 1 1 1 False
Inserting Station: 3109 0 0 1 1 29 1565105755 1 1 1 False
Inserting Station: 3110 0 0 25 0 3 1565108141 1 1 1 False
Inserting Station: 3112 0 0 1 0 32 1565107149 1 1 1 False
Inserting Station: 3113 0 0 15 0 12 1565108256 1 1 1 False
Inserting Station: 3115 0 0 6 1 43 1565108179 1 1 1 False
Inserting Station: 3116 0 0 15 1 19 1565107588 1 1 1 False
Inserting Station: 3117 0 0 21 0 6 1565108265 1 1 1 False
Inserting S

Inserting Station: 3367 0 0 10 1 28 1565107884 1 1 1 True
Inserting Station: 3368 0 0 10 0 20 1565108005 1 1 1 False
Inserting Station: 3369 0 0 27 0 2 1565107547 1 1 1 False
Inserting Station: 3370 0 0 25 0 0 1565101341 1 1 1 False
Inserting Station: 3372 0 0 38 1 0 1565107466 1 1 1 False
Inserting Station: 3373 0 0 2 0 25 1565108097 1 1 1 False
Inserting Station: 3374 0 0 30 0 6 1565108141 1 1 1 True
Inserting Station: 3375 0 0 34 0 1 1565108023 1 1 1 False
Inserting Station: 3376 0 0 37 0 2 1565108069 1 1 1 False
Inserting Station: 3377 0 0 17 0 8 1565107999 1 1 1 False
Inserting Station: 3378 0 0 24 1 3 1565107271 1 1 1 False
Inserting Station: 3379 0 0 13 0 22 1565106389 1 1 1 False
Inserting Station: 3381 0 0 0 1 20 1565107648 1 1 1 False
Inserting Station: 3382 0 0 2 0 29 1565107329 1 1 1 False
Inserting Station: 3383 0 0 25 0 0 1565107427 1 1 1 False
Inserting Station: 3384 0 0 15 0 10 1565106605 1 1 1 False
Inserting Station: 3386 0 0 17 0 10 1565108177 1 1 1 False
Inserting S

Inserting Station: 3582 0 0 20 0 0 1565097648 1 1 1 False
Inserting Station: 3583 0 0 31 0 0 1565107385 1 1 1 False
Inserting Station: 3584 0 0 37 0 5 1565108227 1 1 1 False
Inserting Station: 3585 0 0 21 0 3 1565108254 1 1 1 False
Inserting Station: 3586 0 0 5 0 14 1565097852 1 1 1 False
Inserting Station: 3587 0 0 27 0 0 1565106885 1 1 1 False
Inserting Station: 3588 0 0 1 0 20 1565106693 1 1 1 False
Inserting Station: 3589 0 0 1 0 9 1565107334 1 1 1 False
Inserting Station: 3590 0 0 17 0 2 1565107953 1 1 1 False
Inserting Station: 3591 0 0 4 1 16 1565106803 1 1 1 False
Inserting Station: 3592 0 0 14 0 5 1565107642 1 1 1 False
Inserting Station: 3593 0 0 13 0 22 1565107807 1 1 1 False
Inserting Station: 3595 0 0 18 0 0 1565106058 1 1 1 False
Inserting Station: 3596 0 0 5 0 14 1565107227 1 1 1 False
Inserting Station: 3597 0 0 3 0 0 1565029967 0 0 0 False
Inserting Station: 3598 0 0 15 0 4 1565106983 1 1 1 False
Inserting Station: 3599 0 0 18 0 9 1565108120 1 1 1 False
Inserting Stati

In [9]:
c = db.cursor() # set our cursor 

c.execute("SELECT * FROM StationInfo LIMIT 5;") # look at the first five entries 
rows = c.fetchall()

print(rows)

c.close()
db.commit()

((168, 0, 0, 38, 1, 8, '1564509309', 0, 1, 1, 0), (281, 0, 1, 60, 1, 4, '1564509454', 0, 1, 1, 1), (285, 0, 0, 0, 0, 0, '18001', 0, 0, 1, 1), (304, 0, 0, 20, 1, 12, '1564509353', 0, 1, 1, 1), (337, 0, 0, 31, 1, 5, '1564509023', 0, 1, 1, 0))


## Last but not least, let's set things up so that our database automatically updates every 15 seconds. 

In [10]:
import time 
import json 
import urllib.request
from datetime import datetime

while True:
    
    with urllib.request.urlopen("https://gbfs.citibikenyc.com/gbfs/en/station_status.json") as url:
        data = json.loads(url.read().decode())

    stations = data['data']['stations'] # iterate through the json to find the station data

    db = MySQLdb.connect(passwd="password",db="citibike") # connect to our db

    c = db.cursor() # set our cursor

    query_template = """INSERT IGNORE INTO StationInfo(station_id, num_ebikes_available, num_docks_disabled, \
    num_docks_available, num_bikes_disabled, num_bikes_available, last_reported, is_returning, is_renting, \
    is_installed, eightd_has_available_keys) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""

    for entry in stations: # for every station entry in the json 
        station_id = int(entry['station_id']) # find and set station_id
        num_ebikes_available = int(entry['num_ebikes_available']) # find and set num_ebikes_available
        num_docks_disabled = int(entry['num_docks_disabled']) # find and set num_docks_disabled
        num_docks_available = int(entry['num_docks_available']) # find and set num_docks_available
        num_bikes_disabled = int(entry['num_bikes_disabled']) # find and set num_bikes_disabled
        num_bikes_available = int(entry['num_bikes_available']) # find and set num_bikes_available
        last_reported = str(entry['last_reported']) # find and set last_reported
        is_returning = int(entry['is_returning']) # find and set is_returning
        is_renting = int(entry['is_renting']) # find and set is_renting
        is_installed = int(entry['is_installed']) # find and set is_installed
        eightd_has_available_keys = bool(entry['eightd_has_available_keys']) # find and set eightd_has_available_keys

        c.execute(query_template, query_parameters) # execute insert query and parameters
        
    c.close()
    db.commit()
        
    c = db.cursor() # set our cursor

    c.execute("SELECT * FROM StationInfo LIMIT 5;") # look at the first five entries 
    rows = c.fetchall()

    print(rows)
        
    time.sleep(15)
        


((168, 0, 0, 38, 1, 8, '1564509309', 0, 1, 1, 0), (281, 0, 1, 60, 1, 4, '1564509454', 0, 1, 1, 1), (285, 0, 0, 0, 0, 0, '18001', 0, 0, 1, 1), (304, 0, 0, 20, 1, 12, '1564509353', 0, 1, 1, 1), (337, 0, 0, 31, 1, 5, '1564509023', 0, 1, 1, 0))


KeyboardInterrupt: 

---

## I hope this has helped you find your coding legs! Next week we'll get back to descriptive analytics using Python and Pandas. For now, take time to refresh yourself on the content covered in "Introduction to Programming". 

## If you need a referesher on your SQL skills, check out the "Supplementary Info" directory in the class repo.